In [1]:
from pathlib import Path

import os
import sys

current_path = Path(os.getcwd())
sys.path.append(str(current_path))

In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [5]:
import vectorian

from vectorian.importers import NovelImporter
from vectorian.embeddings import FastText
from vectorian.session import LabSession

embedding = FastText("en")

im = NovelImporter(nlp)
doc = im("/Users/arbeit/A Child's Dream of a Star.txt")

session = LabSession(
    [doc],
    [embedding])

100%|██████████| 58/58 [00:00<00:00, 121.93it/s]


In [7]:
query = nlp("literal")
r = session.find(query, n=3)

FloatProgress(value=0.0, layout=Layout(width='100%'), max=1.0)

In [9]:
r